=============================================================================
# КЛАСС ДЛЯ ОБУЧЕНИЯ И ОЦЕНКИ РЕГРЕССИИ
=============================================================================

In [6]:
# Системные и утилитарные модули
from __future__ import annotations
from dataclasses import dataclass
from typing import Optional, Dict, Any, Tuple

# Работа с данными и модели
import numpy as np  # для загрузки и работы с массивами .npy
from sklearn.linear_model import Ridge  # линейная регрессия с L2-регуляризацией
from sklearn.preprocessing import StandardScaler  # стандартизация признаков
from sklearn.pipeline import Pipeline  # упрощение сборки последовательности трансформаций и модели

# Метрики
from sklearn.metrics import (
    mean_squared_error,        # среднеквадратичная ошибка
    mean_absolute_error,       # средняя абсолютная ошибка
    r2_score,                  # коэффициент детерминации
    explained_variance_score,
)

# Подбор гиперпараметров
from sklearn.model_selection import GridSearchCV, KFold  # CV и подбор сетки параметров

=============================================================================
## ОСНОВНОЙ КЛАСС
=============================================================================

In [7]:
@dataclass
class RidgeTrainer:
    """
    Класс, инкапсулирующий весь процесс:
      1. Загрузка данных из .npy
      2. Создание pipeline: стандартизация + Ridge
      3. GridSearchCV по alpha с KFold
      4. Расчёт метрик на train
      5. Красивый отчёт с результатами

    Атрибуты:
        x_path: путь к файлу с X (.npy)
        y_path: путь к файлу с y (.npy)
        cv_splits: количество фолдов в KFold
        random_state: фиксируем случайность для повторяемости
        n_jobs: сколько потоков использовать для GridSearch
        refit_on: метрика, по которой будет рефититься лучшая модель
    """
    x_path: str
    y_path: str
    cv_splits: int = 5
    random_state: int = 42
    n_jobs: int = -1
    refit_on: str = "neg_mean_squared_error"

    # -------------------------------------------------------------------------
    # Загрузка данных
    # -------------------------------------------------------------------------
    def load_data(self) -> Tuple[np.ndarray, np.ndarray]:
        """
        Загружает данные из указанных .npy файлов.
        Возвращает X и y, где y преобразуется в 1D массив через ravel().
        """
        X = np.load(self.x_path)
        y = np.load(self.y_path).ravel()
        return X, y

    # -------------------------------------------------------------------------
    # Построение pipeline
    # -------------------------------------------------------------------------
    def build_pipeline(self) -> Pipeline:
        """
        Создаёт pipeline:
          - StandardScaler: стандартизируем признаки
          - Ridge: регрессия с L2 регуляризацией
        """
        pipeline = Pipeline(
            steps=[
                ("scaler", StandardScaler()),  # стандартизация
                ("regressor", Ridge()),        # регрессор
            ]
        )
        return pipeline

    # -------------------------------------------------------------------------
    # Подготовка сетки гиперпараметров
    # -------------------------------------------------------------------------
    def make_param_grid(self, start: float = 0.1, stop: float = 15.0, step: float = 0.25) -> Dict[str, Any]:
        """
        Создаёт сетку alpha для Ridge.
        Можно изменить start/stop/step для других диапазонов.
        """
        alphas = np.arange(start, stop, step)
        return {"regressor__alpha": alphas}

    # -------------------------------------------------------------------------
    # Запуск GridSearchCV
    # -------------------------------------------------------------------------
    def run_grid_search(self, X: np.ndarray, y: np.ndarray) -> GridSearchCV:
        """
        Запускает GridSearchCV:
          - pipeline: StandardScaler + Ridge
          - param_grid: alpha
          - KFold с shuffle
          - scoring: MSE, MAE, R2 (refit по выбранной метрике)
        """
        model = self.build_pipeline()
        param_grid = self.make_param_grid()

        cv = KFold(n_splits=self.cv_splits, shuffle=True, random_state=self.random_state)

        scoring = {
            "neg_mean_squared_error": "neg_mean_squared_error",
            "neg_mean_absolute_error": "neg_mean_absolute_error",
            "r2": "r2",
        }

        grid = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            scoring=scoring,
            refit=self.refit_on,
            cv=cv,
            n_jobs=self.n_jobs,
            return_train_score=True,
        )

        grid.fit(X, y)
        return grid

    # -------------------------------------------------------------------------
    # Расчёт метрик на обучающей выборке
    # -------------------------------------------------------------------------
    def evaluate_on_train(self, estimator: Pipeline, X: np.ndarray, y: np.ndarray) -> Dict[str, float]:
        """
        Предсказывает y на X и считает набор метрик:
          - MSE, RMSE, MAE
          - R2
          - Explained Variance
        """
        y_pred = estimator.predict(X)
        mse = mean_squared_error(y, y_pred)
        rmse = mse ** 0.5
        mae = mean_absolute_error(y, y_pred)
        r2 = r2_score(y, y_pred)
        expl_var = explained_variance_score(y, y_pred)

        return {
            "mse": mse,
            "rmse": rmse,
            "mae": mae,
            "r2": r2,
            "explained_variance": expl_var,
        }

    # -------------------------------------------------------------------------
    # Форматирование вывода метрики
    # -------------------------------------------------------------------------
    @staticmethod
    def _format_metric(name: str, value: float, digits: int = 4) -> str:
        """
        Красиво форматирует метрику для вывода.
        """
        return f"{name:20s}: {value:.{digits}f}"

    # -------------------------------------------------------------------------
    # Отчёт о результатах
    # -------------------------------------------------------------------------
    def report(self, grid: GridSearchCV, train_metrics: Dict[str, float]) -> None:
        """
        Печатает компактный, структурированный отчёт:
          - лучший alpha из GridSearch
          - CV метрики (MSE, RMSE)
          - метрики на train (MSE, RMSE, MAE, R2, Explained Variance)
        """
        best_alpha = grid.best_params_.get("regressor__alpha")
        best_mse_cv = -grid.best_score_  # sklearn возвращает отрицательное MSE
        best_rmse_cv = best_mse_cv ** 0.5

        print("\n" + "=" * 70)
        print("РЕЗУЛЬТАТЫ GRID SEARCH (CV)".center(70))
        print("=" * 70)
        print(f"Лучший параметр alpha: {best_alpha}")
        print(self._format_metric("CV MSE", best_mse_cv, digits=4))
        print(self._format_metric("CV RMSE", best_rmse_cv, digits=4))

        print("\n" + "-" * 70)
        print("МЕТРИКИ НА TRAIN (для сравнения)".center(70))
        print("-" * 70)
        for k, v in train_metrics.items():
            digits = 4 if k in ("r2", "explained_variance") else 2
            print(self._format_metric(k.upper(), v, digits=digits))

        print("=" * 70 + "\n")

    # -------------------------------------------------------------------------
    # Главный метод для запуска всего процесса
    # -------------------------------------------------------------------------
    def run(self) -> dict[str, Any]:
        """
        1. Загружает данные
        2. Запускает GridSearchCV
        3. Вычисляет метрики на train
        4. Выводит отчёт
        5. Возвращает словарь с результатами
        """
        X, y = self.load_data()
        grid = self.run_grid_search(X, y)
        best_model = grid.best_estimator_
        train_metrics = self.evaluate_on_train(best_model, X, y)
        self.report(grid, train_metrics)

        return {
            "grid_search": grid,
            "best_model": best_model,
            "train_metrics": train_metrics,
        }

=============================================================================
# ИСПОЛЬЗОВАНИЕ КЛАССА
=============================================================================

In [10]:
trainer = RidgeTrainer(
    x_path="data/X.npy",
    y_path="data/y.npy",
    cv_splits=5,
    random_state=42,
    n_jobs=-1,
    refit_on="neg_mean_squared_error",
)
results = trainer.run()


                     РЕЗУЛЬТАТЫ GRID SEARCH (CV)                      
Лучший параметр alpha: 14.849999999999998
CV MSE              : 1615056089.7886
CV RMSE             : 40187.7604

----------------------------------------------------------------------
                   МЕТРИКИ НА TRAIN (для сравнения)                   
----------------------------------------------------------------------
MSE                 : 1612899742.79
RMSE                : 40160.92
MAE                 : 30704.33
R2                  : 0.4181
EXPLAINED_VARIANCE  : 0.4181

